# Yfinance_Usecase 
- developing python definition funneled into langchain tool thus to be selected  

In [1]:
from typing import Type
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

from datetime import datetime, timedelta

import yfinance as yf

In [14]:
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="6m")
    return {"price": recent.iloc[0]["Close"], \
            #"currency": ticker_data.info["currency"]
            }


def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")



In [21]:
class StockPerformanceInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="days duration since the given date to the latest trading day")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to get the stock performance over the given period.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = StockPerformanceInput

    def _run(self, ticker: str , days: int):
        performance_response = get_stock_performance(ticker,days)
        return performance_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [44]:
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="")

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)
tools = [CurrentStockPriceTool(),StockPerformanceTool(),tavily_tool
]

agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("get me 3 data [the latest trading day stock price of Samsung Electronics(KRX), the date in which \
          Korea banned short-selling, its performance since the ban].\n"
          "Do not assume any random dates if the article searched doesn't say about it, you can tell me\
            that article's release date and ultimately the performance since then")



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}